In [1]:
import numpy as np
from numpy import argmax
import pandas as pd
from collections import Counter
from datetime import datetime
import os
import itertools
import math
import collections

import requests
import spotipy
import spotipy.util as util
import time

from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials

import warnings 
warnings.filterwarnings("ignore")

from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict

pd.set_option('display.max_columns', None)


In [2]:
client_id = 'f535866be6e040df8c067cbacc45aaf0' 
client_secret = '035254dba5c04de8887abfba594f5bcd'
redirect_uri = 'http://localhost:3000'
scope = "user-library-read playlist-read-private"

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope
))

In [4]:

# Fetch all liked songs
liked_songs = []
limit = 50
offset = 0

while True:
    results = sp.current_user_saved_tracks(limit=limit, offset=offset)
    liked_songs.extend(results['items'])

    if len(results['items']) < limit:
        break
    offset += limit

# Print all liked songs
for idx, item in enumerate(liked_songs):
    track = item['track']
    print(f"{idx + 1}: {track['id']}: {track['name']} by {', '.join(artist['name'] for artist in track['artists'])}")


1: 7Kzy7StKclwXPJWFIJ7l1X: In The Living Room by Maggie Rogers
2: 5V6mK1pEu22104f22m8KkX: I Told You Things by Gracie Abrams
3: 51hyZpbJlIgZIaX3TtMxmu: run for the hills by Tate McRae
4: 2vBuFCT5HWuz8oi0C4Fr05: AMANECIÓ by Quevedo, De La Rose, De La Ghetto
5: 2NFadq6pUeiVEihLvUlOSr: Rauw Alejandro: Bzrp Music Sessions, Vol. 56 by Bizarrap, Rauw Alejandro
6: 3A1BJKGMsa8JqO8M8zztyq: Me Gustas Tu by Manu Chao
7: 2lwwrWVKdf3LR9lbbhnr6R: Float On by Modest Mouse
8: 77RNVzAVwqWiZrZuWqMV2i: Nancy From Now On by Father John Misty
9: 6uW4QhvYTl6q24OBbWTt9s: Convite by Lamparina
10: 0nrlsriZeFXhuFno9bVLNg: secrecy by Ethan Regan
11: 5vbnQr1tWSSREmcIONIUm9: Backwards Directions by Abby Sage
12: 2FwDApgXk91kXvqy2oB7dz: Simulation Swarm by Big Thief
13: 04mAOoQNsXmDJlyupJwmkO: Waiting Room by Phoebe Bridgers
14: 7CoMBpPTwQi2wPT0U0Nr9b: State Lines by Novo Amor
15: 2xFmXeAKtzrG1ZQBT6adKc: Keep The Rain by Searows
16: 6GyFQqsD80Kwdna2jWHyjY: Settle Down by Palace
17: 33xgQAHpWcbY8B45sakPCn: Tastes Li

In [5]:
# Extract track data
track_data = []
for item in liked_songs:
    track = item['track']
    track_data.append({
        'id': track['id'],
        'name': track['name'],
        'artists': ', '.join(artist['name'] for artist in track['artists']),
        'album': track['album']['name'],
        'release_date': track['album']['release_date'],
        'popularity': track['popularity']
    })


In [6]:
df_tracks = pd.DataFrame(track_data)
df_tracks.head()

top_100 = df_tracks.head(100)
top_100

,id,name,artists,album,release_date,popularity
0,7Kzy7StKclwXPJWFIJ7l1X,In The Living Room,Maggie Rogers,In The Living Room,2024-10-08,63
1,5V6mK1pEu22104f22m8KkX,I Told You Things,Gracie Abrams,The Secret of Us (Deluxe),2024-10-18,74
2,51hyZpbJlIgZIaX3TtMxmu,run for the hills,Tate McRae,THINK LATER,2023-12-08,80
3,2vBuFCT5HWuz8oi0C4Fr05,AMANECIÓ,"Quevedo, De La Rose, De La Ghetto",AMANECIÓ,2024-11-20,53
4,2NFadq6pUeiVEihLvUlOSr,"Rauw Alejandro: Bzrp Music Sessions, Vol. 56","Bizarrap, Rauw Alejandro","Rauw Alejandro: Bzrp Music Sessions, Vol. 56",2023-06-21,65
...,...,...,...,...,...,...
95,2d8UxVNhJinc8uat9PoM9y,Who’s Afraid of Little Old Me?,Taylor Swift,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,2024-04-19,74
96,1xtw1krCR6Dw2KwkXw5z63,The Smallest Man Who Ever Lived,Taylor Swift,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,2024-04-19,74
97,18WFFUIsewmA8g31KAeo3e,The Prophecy,Taylor Swift,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,2024-04-19,76
98,2913xXOVAIDAqxzV2g4VcU,The Bolter,Taylor Swift,THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY,2024-04-19,72


In [7]:
song_ids = top_100['id'].tolist()
audio_features = sp.audio_features(song_ids)

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7Kzy7StKclwXPJWFIJ7l1X,5V6mK1pEu22104f22m8KkX,51hyZpbJlIgZIaX3TtMxmu,2vBuFCT5HWuz8oi0C4Fr05,2NFadq6pUeiVEihLvUlOSr,3A1BJKGMsa8JqO8M8zztyq,2lwwrWVKdf3LR9lbbhnr6R,77RNVzAVwqWiZrZuWqMV2i,6uW4QhvYTl6q24OBbWTt9s,0nrlsriZeFXhuFno9bVLNg,5vbnQr1tWSSREmcIONIUm9,2FwDApgXk91kXvqy2oB7dz,04mAOoQNsXmDJlyupJwmkO,7CoMBpPTwQi2wPT0U0Nr9b,2xFmXeAKtzrG1ZQBT6adKc,6GyFQqsD80Kwdna2jWHyjY,33xgQAHpWcbY8B45sakPCn,0RlNltpVEnGcGD8srnSHSS,5PNAsZO4JyT8fUzPyKwZ7W,2krfD3TzLiV1WVNepAYJW9,2le9fblYnfoLr9dkZIsJUa,4faTuWPQeMo24zvuTDre3c,1SerYv7SabsGxVl0qIKES9,1H3vHQSD9528EfWpoXaQik,6YTBfxkdGce9pLEy9jG9Jo,3ej9CjmXF3FvJDmWP0us2m,6kvdjKkkd2QJ0arXpjCmfW,1awQLv1J6LkeEQ5co4OOoQ,0uhYgQQ6WX9rCONnum7Fmw,2Ij4PtXhIlI2uIP4slvnUA,562WsbcGbzCzvIoMWusiQ0,1x1z2qHf13S4nD4ozEEH02,3jDdpx9PMlfMBS5tOBHFm9,4l0HxP8wm5iWHqo9myvFsm,5WphWTUIfRe7x8NZss79cY,6Jv7kjGkhY2fT4yuBF3aTz,6XK6Zw6JkFsHXzAcMWNiIr,31TAub5WKWEsVTJcdksxq7,7H8Ga5vDJKhgxQ0uTiScUf,1OFKUn2VLafrHj7ybnap0Q,2NtqI3Ylig7

SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=7Kzy7StKclwXPJWFIJ7l1X,5V6mK1pEu22104f22m8KkX,51hyZpbJlIgZIaX3TtMxmu,2vBuFCT5HWuz8oi0C4Fr05,2NFadq6pUeiVEihLvUlOSr,3A1BJKGMsa8JqO8M8zztyq,2lwwrWVKdf3LR9lbbhnr6R,77RNVzAVwqWiZrZuWqMV2i,6uW4QhvYTl6q24OBbWTt9s,0nrlsriZeFXhuFno9bVLNg,5vbnQr1tWSSREmcIONIUm9,2FwDApgXk91kXvqy2oB7dz,04mAOoQNsXmDJlyupJwmkO,7CoMBpPTwQi2wPT0U0Nr9b,2xFmXeAKtzrG1ZQBT6adKc,6GyFQqsD80Kwdna2jWHyjY,33xgQAHpWcbY8B45sakPCn,0RlNltpVEnGcGD8srnSHSS,5PNAsZO4JyT8fUzPyKwZ7W,2krfD3TzLiV1WVNepAYJW9,2le9fblYnfoLr9dkZIsJUa,4faTuWPQeMo24zvuTDre3c,1SerYv7SabsGxVl0qIKES9,1H3vHQSD9528EfWpoXaQik,6YTBfxkdGce9pLEy9jG9Jo,3ej9CjmXF3FvJDmWP0us2m,6kvdjKkkd2QJ0arXpjCmfW,1awQLv1J6LkeEQ5co4OOoQ,0uhYgQQ6WX9rCONnum7Fmw,2Ij4PtXhIlI2uIP4slvnUA,562WsbcGbzCzvIoMWusiQ0,1x1z2qHf13S4nD4ozEEH02,3jDdpx9PMlfMBS5tOBHFm9,4l0HxP8wm5iWHqo9myvFsm,5WphWTUIfRe7x8NZss79cY,6Jv7kjGkhY2fT4yuBF3aTz,6XK6Zw6JkFsHXzAcMWNiIr,31TAub5WKWEsVTJcdksxq7,7H8Ga5vDJKhgxQ0uTiScUf,1OFKUn2VLa

In [24]:
track_ids = [track['id'] for track in track_data[:100]]
audio_features = sp.audio_features(track_ids)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7Kzy7StKclwXPJWFIJ7l1X,5V6mK1pEu22104f22m8KkX,51hyZpbJlIgZIaX3TtMxmu,2vBuFCT5HWuz8oi0C4Fr05,2NFadq6pUeiVEihLvUlOSr,3A1BJKGMsa8JqO8M8zztyq,2lwwrWVKdf3LR9lbbhnr6R,77RNVzAVwqWiZrZuWqMV2i,6uW4QhvYTl6q24OBbWTt9s,0nrlsriZeFXhuFno9bVLNg,5vbnQr1tWSSREmcIONIUm9,2FwDApgXk91kXvqy2oB7dz,04mAOoQNsXmDJlyupJwmkO,7CoMBpPTwQi2wPT0U0Nr9b,2xFmXeAKtzrG1ZQBT6adKc,6GyFQqsD80Kwdna2jWHyjY,33xgQAHpWcbY8B45sakPCn,0RlNltpVEnGcGD8srnSHSS,5PNAsZO4JyT8fUzPyKwZ7W,2krfD3TzLiV1WVNepAYJW9,2le9fblYnfoLr9dkZIsJUa,4faTuWPQeMo24zvuTDre3c,1SerYv7SabsGxVl0qIKES9,1H3vHQSD9528EfWpoXaQik,6YTBfxkdGce9pLEy9jG9Jo,3ej9CjmXF3FvJDmWP0us2m,6kvdjKkkd2QJ0arXpjCmfW,1awQLv1J6LkeEQ5co4OOoQ,0uhYgQQ6WX9rCONnum7Fmw,2Ij4PtXhIlI2uIP4slvnUA,562WsbcGbzCzvIoMWusiQ0,1x1z2qHf13S4nD4ozEEH02,3jDdpx9PMlfMBS5tOBHFm9,4l0HxP8wm5iWHqo9myvFsm,5WphWTUIfRe7x8NZss79cY,6Jv7kjGkhY2fT4yuBF3aTz,6XK6Zw6JkFsHXzAcMWNiIr,31TAub5WKWEsVTJcdksxq7,7H8Ga5vDJKhgxQ0uTiScUf,1OFKUn2VLafrHj7ybnap0Q,2NtqI3Ylig7

SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=7Kzy7StKclwXPJWFIJ7l1X,5V6mK1pEu22104f22m8KkX,51hyZpbJlIgZIaX3TtMxmu,2vBuFCT5HWuz8oi0C4Fr05,2NFadq6pUeiVEihLvUlOSr,3A1BJKGMsa8JqO8M8zztyq,2lwwrWVKdf3LR9lbbhnr6R,77RNVzAVwqWiZrZuWqMV2i,6uW4QhvYTl6q24OBbWTt9s,0nrlsriZeFXhuFno9bVLNg,5vbnQr1tWSSREmcIONIUm9,2FwDApgXk91kXvqy2oB7dz,04mAOoQNsXmDJlyupJwmkO,7CoMBpPTwQi2wPT0U0Nr9b,2xFmXeAKtzrG1ZQBT6adKc,6GyFQqsD80Kwdna2jWHyjY,33xgQAHpWcbY8B45sakPCn,0RlNltpVEnGcGD8srnSHSS,5PNAsZO4JyT8fUzPyKwZ7W,2krfD3TzLiV1WVNepAYJW9,2le9fblYnfoLr9dkZIsJUa,4faTuWPQeMo24zvuTDre3c,1SerYv7SabsGxVl0qIKES9,1H3vHQSD9528EfWpoXaQik,6YTBfxkdGce9pLEy9jG9Jo,3ej9CjmXF3FvJDmWP0us2m,6kvdjKkkd2QJ0arXpjCmfW,1awQLv1J6LkeEQ5co4OOoQ,0uhYgQQ6WX9rCONnum7Fmw,2Ij4PtXhIlI2uIP4slvnUA,562WsbcGbzCzvIoMWusiQ0,1x1z2qHf13S4nD4ozEEH02,3jDdpx9PMlfMBS5tOBHFm9,4l0HxP8wm5iWHqo9myvFsm,5WphWTUIfRe7x8NZss79cY,6Jv7kjGkhY2fT4yuBF3aTz,6XK6Zw6JkFsHXzAcMWNiIr,31TAub5WKWEsVTJcdksxq7,7H8Ga5vDJKhgxQ0uTiScUf,1OFKUn2VLa

In [30]:
audio_data = pd.DataFrame(audio_features)
audio_data.head()

NameError: name 'audio_features' is not defined

In [39]:
merged_df = pd.merge(df_tracks, audio_data, on='id', how='inner')
merged_df.head()
liked_songs = merged_df
#liked_songs.to_csv('liked_songs.csv', index=False)

In [42]:
liked_songs.shape
liked_songs

,id,name,artists,album,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2NFadq6pUeiVEihLvUlOSr,"Rauw Alejandro: Bzrp Music Sessions, Vol. 56","Bizarrap, Rauw Alejandro","Rauw Alejandro: Bzrp Music Sessions, Vol. 56",2023-06-21,65,0.774,0.572,11,-6.000,1,0.0506,0.3000,0.003260,0.3150,0.489,128.010,audio_features,spotify:track:2NFadq6pUeiVEihLvUlOSr,https://api.spotify.com/v1/tracks/2NFadq6pUeiV...,https://api.spotify.com/v1/audio-analysis/2NFa...,191254,4
1,3A1BJKGMsa8JqO8M8zztyq,Me Gustas Tu,Manu Chao,Me Gustas Tu EP,2023-02-13,59,0.657,0.742,11,-6.822,0,0.0827,0.6620,0.493000,0.2220,0.909,175.943,audio_features,spotify:track:3A1BJKGMsa8JqO8M8zztyq,https://api.spotify.com/v1/tracks/3A1BJKGMsa8J...,https://api.spotify.com/v1/audio-analysis/3A1B...,239987,4
2,2lwwrWVKdf3LR9lbbhnr6R,Float On,Modest Mouse,Good News For People Who Love Bad News,2004-04-05,71,0.649,0.888,6,-4.807,1,0.0293,0.0130,0.000002,0.0888,0.553,100.975,audio_features,spotify:track:2lwwrWVKdf3LR9lbbhnr6R,https://api.spotify.com/v1/tracks/2lwwrWVKdf3L...,https://api.spotify.com/v1/audio-analysis/2lww...,208467,4
3,77RNVzAVwqWiZrZuWqMV2i,Nancy From Now On,Father John Misty,Fear Fun,2012-05-01,61,0.535,0.646,0,-9.168,1,0.0278,0.5260,0.040600,0.0908,0.501,107.078,audio_features,spotify:track:77RNVzAVwqWiZrZuWqMV2i,https://api.spotify.com/v1/tracks/77RNVzAVwqWi...,https://api.spotify.com/v1/audio-analysis/77RN...,234160,4
4,6uW4QhvYTl6q24OBbWTt9s,Convite,Lamparina,Manda Dizer,2018-08-06,44,0.470,0.621,11,-7.369,0,0.0620,0.5900,0.000000,0.1270,0.618,117.952,audio_features,spotify:track:6uW4QhvYTl6q24OBbWTt9s,https://api.spotify.com/v1/tracks/6uW4QhvYTl6q...,https://api.spotify.com/v1/audio-analysis/6uW4...,213384,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5xtxxuIqInrLDSfjA6Co3o,Fiebre (Prod. King DouDou),Bad Gyal,Slow Wine Mixtape,2016-11-11,71,0.421,0.720,7,-6.257,1,0.0571,0.2660,0.000000,0.2250,0.240,97.998,audio_features,spotify:track:5xtxxuIqInrLDSfjA6Co3o,https://api.spotify.com/v1/tracks/5xtxxuIqInrL...,https://api.spotify.com/v1/audio-analysis/5xtx...,247500,4
96,0DcfDOaYG8VSZW9ztShlms,Roll My Stone,Arcy Drive,Roll My Stone,2022-07-15,47,0.494,0.858,0,-3.947,1,0.0457,0.0839,0.000000,0.1080,0.374,98.478,audio_features,spotify:track:0DcfDOaYG8VSZW9ztShlms,https://api.spotify.com/v1/tracks/0DcfDOaYG8VS...,https://api.spotify.com/v1/audio-analysis/0Dcf...,179860,3
97,3xdjjKMcMOFgo1eQrfbogM,Wagon Wheel,Darius Rucker,True Believers,2013-01-01,75,0.535,0.660,9,-7.174,1,0.0297,0.4550,0.000000,0.1350,0.577,148.044,audio_features,spotify:track:3xdjjKMcMOFgo1eQrfbogM,https://api.spotify.com/v1/tracks/3xdjjKMcMOFg...,https://api.spotify.com/v1/audio-analysis/3xdj...,298373,4
98,4DoEHKWh8k6vYDleECQkJx,Los Tontos,"C. Tangana, Kiko Veneno",El Madrileño,2021-02-26,59,0.798,0.715,9,-5.712,0,0.1870,0.3650,0.000002,0.1420,0.895,105.995,audio_features,spotify:track:4DoEHKWh8k6vYDleECQkJx,https://api.spotify.com/v1/tracks/4DoEHKWh8k6v...,https://api.spotify.com/v1/audio-analysis/4DoE...,192787,4


In [8]:
playlist_id = '4WBtYgr7OTuqoR2bMPUvOe'  

all_tracks = []

limit = 100  # Spotify allows a maximum of 100 tracks per request
offset = 0

while True:
    # Fetch a batch of tracks
    results = sp.playlist_tracks(playlist_id, limit=limit, offset=offset)

    # Extract track info and add it to the all_tracks list
    all_tracks.extend(results['items'])

    # If the number of tracks in the response is less than the limit, stop
    if len(results['items']) < limit:
        break

    # Otherwise, move to the next page of tracks
    offset += limit

# Print track details
for idx, item in enumerate(all_tracks):
    track = item['track']
    print(f"{idx + 1}: {track['id']}: {track['name']} by {', '.join(artist['name'] for artist in track['artists'])}")


1: 2EjXfH91m7f8HiJN1yQg97: Rockin' Around The Christmas Tree by Brenda Lee
2: 7vQbuQcyTflfCIOu3Uzzya: Jingle Bell Rock by Bobby Helms
3: 5hslUAKq9I9CG2bAulFkHN: It's the Most Wonderful Time of the Year by Andy Williams
4: 46pF1zFimM582ss1PrMy68: Christmas (Baby Please Come Home) by Darlene Love
5: 4PS1e8f2LvuTFgUs1Cn3ON: The Christmas Song (Merry Christmas To You) by Nat King Cole
6: 1foCxQtxBweJtZmdxhEHVO: Santa Baby (with Henri René & His Orchestra) by Eartha Kitt, Henri René and His Orchestra
7: 3XsaSIPWvM61RIFfeb0BBR: White Christmas - 1947 Version by Bing Crosby, Ken Darby Singers, John Scott Trotter & His Orchestra
8: 0oPdaY4dXtc3ZsaG17V972: Feliz Navidad by José Feliciano
9: 2uFaJJtFpPDc5Pa95XzTvg: Let It Snow! Let It Snow! Let It Snow! by Dean Martin
10: 77khP2fIVhSW23NwxrRluh: A Holly Jolly Christmas by Burl Ives
11: 5ASM6Qjiav2xPe7gRkQMsQ: Sleigh Ride by The Ronettes
12: 030mot3ZKR3oskfMsqDB2R: Have Yourself A Merry Little Christmas by Judy Garland
13: 2pXpURmn6zC5ZYDMms6fwa:

In [9]:
track_data_xmas = []
for item in all_tracks:
    track = item['track']
    track_data_xmas.append({
        'id': track['id'],
        'name': track['name'],
        'artists': ', '.join(artist['name'] for artist in track['artists']),
        'album': track['album']['name'],
        'release_date': track['album']['release_date'],
        'popularity': track['popularity']
    })

In [10]:
xmas = pd.DataFrame(track_data_xmas)
xmas

,id,name,artists,album,release_date,popularity
0,2EjXfH91m7f8HiJN1yQg97,Rockin' Around The Christmas Tree,Brenda Lee,Merry Christmas From Brenda Lee,1964-10-19,89
1,7vQbuQcyTflfCIOu3Uzzya,Jingle Bell Rock,Bobby Helms,Jingle Bell Rock/Captain Santa Claus (And His ...,1957-12-02,85
2,5hslUAKq9I9CG2bAulFkHN,It's the Most Wonderful Time of the Year,Andy Williams,The Andy Williams Christmas Album,1963-11-24,86
3,46pF1zFimM582ss1PrMy68,Christmas (Baby Please Come Home),Darlene Love,A Christmas Gift For You From Phil Spector,1963,72
4,4PS1e8f2LvuTFgUs1Cn3ON,The Christmas Song (Merry Christmas To You),Nat King Cole,The Christmas Song (Expanded Edition),1962,81
...,...,...,...,...,...,...
1957,1C6bP1OpwtZO2RDKlQfFtW,Prende el Arbolito - Spotify Singles Holiday,"El Alfa, El Cherry Scom, Kiko el Crazy, Shelow...",Prende el Arbolito - Spotify Singles Holiday,2021-11-17,23
1958,2ScUNIrBDPOR1bj36Z7QjC,Shchedrivka - Spotify Singles Holiday,Kalush Orchestra,Spotify Singles Holiday,2022-11-16,24
1959,04LnqfQEzvutfUhSylWowv,Must Be Santa - Spotify Singles Holiday,Kurt Vile,Must Be Santa (Spotify Singles Holiday),2022-11-16,31
1960,5RPVSzZW6R0pUza8Hs2sC7,It's the Most Wonderful Time of the Year,Andy Williams,The Classic Christmas Album,2013-09-27,49


In [11]:
def get_audio_features_in_batches(track_ids):
    audio_features = []
    # Split the list of track IDs into chunks of 100
    for i in range(0, len(track_ids), 100):
        chunk = track_ids[i:i + 100]  # Get the next 100 track IDs
        features = sp.audio_features(chunk)  # Fetch the audio features for this chunk
        
        # Filter out any None values (tracks without audio features)
        features = [feature for feature in features if feature is not None]
        
        audio_features.extend(features)  # Add the features to the list
    
    return audio_features

audio_features_list = get_audio_features_in_batches(xmas['id'])

audio_xmas = pd.DataFrame(audio_features_list)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2EjXfH91m7f8HiJN1yQg97,7vQbuQcyTflfCIOu3Uzzya,5hslUAKq9I9CG2bAulFkHN,46pF1zFimM582ss1PrMy68,4PS1e8f2LvuTFgUs1Cn3ON,1foCxQtxBweJtZmdxhEHVO,3XsaSIPWvM61RIFfeb0BBR,0oPdaY4dXtc3ZsaG17V972,2uFaJJtFpPDc5Pa95XzTvg,77khP2fIVhSW23NwxrRluh,5ASM6Qjiav2xPe7gRkQMsQ,030mot3ZKR3oskfMsqDB2R,2pXpURmn6zC5ZYDMms6fwa,5P8Xvy5tzhmfyfA6GplQ8h,75dfH68JDisE8dDaD4KlVY,4HEOgBHRCExyYVeTyrXsnL,4qP2V09IpTct5A1ZSnr1zh,3QiAAp20rPC3dcAtKtMaqQ,25leEEaz1gIpp7o21Fqyjo,04vLj9QUXoKdRlsp3gkURo,12YAgUbl6Uk9E7fzopF4Ji,3QIoEi8Enr9uHffwInGIsC,38xhBO2AKrJnjdjVnhJES6,6Uekkm0MOVT4WznQy0ktVR,2QWKbE6GpWhRtlbRwcP7g8,7GNsHkiYPcQQjvhTiILFUL,0rHToGels2lt8Y0mCYoF90,7azp0ySqPkkXHi9M9aeg3T,3bMfUUpzUzqWp9II00yRhE,7n5m8nDAnyXo81tr4B3Bcw,11Z7sdkpC5IaaydoIvDn74,5aDoUmxBsbdpS16alksb5Z,3sDdyBHQ60Cs1opmIyRvhp,0z5HSQGWgnjdNvwbOmhB2w,7ehhnVu3JUWMaSJzY1zC4k,2GapxG7BxK55ihQRAlR39e,6syO6UTl8QiiWADJy3Uy2d,4YhY2qHdBmSX33uj8ms0oq,47otoIkhx3fkdivEXL5OB6,1KIJSvzRZzo8ehDTPOXKru,0H1zX27yxoD

SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=2EjXfH91m7f8HiJN1yQg97,7vQbuQcyTflfCIOu3Uzzya,5hslUAKq9I9CG2bAulFkHN,46pF1zFimM582ss1PrMy68,4PS1e8f2LvuTFgUs1Cn3ON,1foCxQtxBweJtZmdxhEHVO,3XsaSIPWvM61RIFfeb0BBR,0oPdaY4dXtc3ZsaG17V972,2uFaJJtFpPDc5Pa95XzTvg,77khP2fIVhSW23NwxrRluh,5ASM6Qjiav2xPe7gRkQMsQ,030mot3ZKR3oskfMsqDB2R,2pXpURmn6zC5ZYDMms6fwa,5P8Xvy5tzhmfyfA6GplQ8h,75dfH68JDisE8dDaD4KlVY,4HEOgBHRCExyYVeTyrXsnL,4qP2V09IpTct5A1ZSnr1zh,3QiAAp20rPC3dcAtKtMaqQ,25leEEaz1gIpp7o21Fqyjo,04vLj9QUXoKdRlsp3gkURo,12YAgUbl6Uk9E7fzopF4Ji,3QIoEi8Enr9uHffwInGIsC,38xhBO2AKrJnjdjVnhJES6,6Uekkm0MOVT4WznQy0ktVR,2QWKbE6GpWhRtlbRwcP7g8,7GNsHkiYPcQQjvhTiILFUL,0rHToGels2lt8Y0mCYoF90,7azp0ySqPkkXHi9M9aeg3T,3bMfUUpzUzqWp9II00yRhE,7n5m8nDAnyXo81tr4B3Bcw,11Z7sdkpC5IaaydoIvDn74,5aDoUmxBsbdpS16alksb5Z,3sDdyBHQ60Cs1opmIyRvhp,0z5HSQGWgnjdNvwbOmhB2w,7ehhnVu3JUWMaSJzY1zC4k,2GapxG7BxK55ihQRAlR39e,6syO6UTl8QiiWADJy3Uy2d,4YhY2qHdBmSX33uj8ms0oq,47otoIkhx3fkdivEXL5OB6,1KIJSvzRZz

In [14]:
audio_xmas.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.598,0.470,8,-8.744,1,0.0496,0.617,0.000000,0.5050,0.879,67.086,audio_features,2EjXfH91m7f8HiJN1yQg97,spotify:track:2EjXfH91m7f8HiJN1yQg97,https://api.spotify.com/v1/tracks/2EjXfH91m7f8...,https://api.spotify.com/v1/audio-analysis/2EjX...,126267,4
1,0.754,0.424,2,-8.463,1,0.0363,0.643,0.000000,0.0652,0.806,119.705,audio_features,7vQbuQcyTflfCIOu3Uzzya,spotify:track:7vQbuQcyTflfCIOu3Uzzya,https://api.spotify.com/v1/tracks/7vQbuQcyTflf...,https://api.spotify.com/v1/audio-analysis/7vQb...,130973,4
2,0.240,0.598,7,-8.435,1,0.0369,0.766,0.000000,0.1170,0.776,201.629,audio_features,5hslUAKq9I9CG2bAulFkHN,spotify:track:5hslUAKq9I9CG2bAulFkHN,https://api.spotify.com/v1/tracks/5hslUAKq9I9C...,https://api.spotify.com/v1/audio-analysis/5hsl...,151933,3
3,0.325,0.759,8,-7.146,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,audio_features,46pF1zFimM582ss1PrMy68,spotify:track:46pF1zFimM582ss1PrMy68,https://api.spotify.com/v1/tracks/46pF1zFimM58...,https://api.spotify.com/v1/audio-analysis/46pF...,166453,4
4,0.320,0.210,6,-15.231,1,0.0345,0.920,0.000080,0.1380,0.208,73.196,audio_features,4PS1e8f2LvuTFgUs1Cn3ON,spotify:track:4PS1e8f2LvuTFgUs1Cn3ON,https://api.spotify.com/v1/tracks/4PS1e8f2LvuT...,https://api.spotify.com/v1/audio-analysis/4PS1...,192160,5


In [38]:
merged_xmas = pd.merge(xmas, audio_xmas, on='id', how='inner')
merged_xmas.head()
#merged_xmas.to_csv('xmas_data.csv', index=False)

,id,name,artists,album,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2EjXfH91m7f8HiJN1yQg97,Rockin' Around The Christmas Tree,Brenda Lee,Merry Christmas From Brenda Lee,1964-10-19,83,0.598,0.470,8,-8.744,1,0.0496,0.617,0.000000,0.5050,0.879,67.086,audio_features,spotify:track:2EjXfH91m7f8HiJN1yQg97,https://api.spotify.com/v1/tracks/2EjXfH91m7f8...,https://api.spotify.com/v1/audio-analysis/2EjX...,126267,4
1,7vQbuQcyTflfCIOu3Uzzya,Jingle Bell Rock,Bobby Helms,Jingle Bell Rock/Captain Santa Claus (And His ...,1957-12-02,80,0.754,0.424,2,-8.463,1,0.0363,0.643,0.000000,0.0652,0.806,119.705,audio_features,spotify:track:7vQbuQcyTflfCIOu3Uzzya,https://api.spotify.com/v1/tracks/7vQbuQcyTflf...,https://api.spotify.com/v1/audio-analysis/7vQb...,130973,4
2,5hslUAKq9I9CG2bAulFkHN,It's the Most Wonderful Time of the Year,Andy Williams,The Andy Williams Christmas Album,1963-11-24,80,0.240,0.598,7,-8.435,1,0.0369,0.766,0.000000,0.1170,0.776,201.629,audio_features,spotify:track:5hslUAKq9I9CG2bAulFkHN,https://api.spotify.com/v1/tracks/5hslUAKq9I9C...,https://api.spotify.com/v1/audio-analysis/5hsl...,151933,3
3,46pF1zFimM582ss1PrMy68,Christmas (Baby Please Come Home),Darlene Love,A Christmas Gift For You From Phil Spector,1963,66,0.325,0.759,8,-7.146,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,audio_features,spotify:track:46pF1zFimM582ss1PrMy68,https://api.spotify.com/v1/tracks/46pF1zFimM58...,https://api.spotify.com/v1/audio-analysis/46pF...,166453,4
4,4PS1e8f2LvuTFgUs1Cn3ON,The Christmas Song (Merry Christmas To You),Nat King Cole,The Christmas Song (Expanded Edition),1962,75,0.320,0.210,6,-15.231,1,0.0345,0.920,0.000080,0.1380,0.208,73.196,audio_features,spotify:track:4PS1e8f2LvuTFgUs1Cn3ON,https://api.spotify.com/v1/tracks/4PS1e8f2LvuT...,https://api.spotify.com/v1/audio-analysis/4PS1...,192160,5


In [16]:
merged_xmas.shape

(1960, 23)

In [63]:
popularity_avg = liked_songs['popularity'].mean()
danceability_avg = liked_songs['danceability'].mean()
key_avg = liked_songs['key'].mean()
loudness_avg = liked_songs['loudness'].mean()
mode_avg = liked_songs['mode'].mean()
speechiness_avg = liked_songs['speechiness'].mean()
acousticness_avg = liked_songs['acousticness'].mean()
instrumentalness_avg = liked_songs['instrumentalness'].mean()
liveness_avg = liked_songs['liveness'].mean()
valence_avg = liked_songs['valence'].mean()
tempo_avg = liked_songs['tempo'].mean()
duration_avg = liked_songs['duration_ms'].mean()
time_avg = liked_songs['time_signature'].mean()

In [64]:
user_vector = [popularity_avg,danceability_avg,key_avg,loudness_avg,mode_avg,speechiness_avg,acousticness_avg,
                instrumentalness_avg,liveness_avg,valence_avg,tempo_avg,duration_avg,time_avg]

In [65]:
user_vector

[58.63,
 0.59393,
 5.21,
 -8.726320000000001,
 0.71,
 0.05485800000000001,
 0.36146888,
 0.09866979729999999,
 0.150994,
 0.50223,
 119.84297999999997,
 216932.31,
 3.9]

In [66]:
xmas_matrix = merged_xmas.drop(['artists', 'album','release_date','type','uri','track_href','analysis_url'], axis=1)
xmas_matrix.head()

,id,name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2EjXfH91m7f8HiJN1yQg97,Rockin' Around The Christmas Tree,83,0.598,0.470,8,-8.744,1,0.0496,0.617,0.000000,0.5050,0.879,67.086,126267,4
1,7vQbuQcyTflfCIOu3Uzzya,Jingle Bell Rock,80,0.754,0.424,2,-8.463,1,0.0363,0.643,0.000000,0.0652,0.806,119.705,130973,4
2,5hslUAKq9I9CG2bAulFkHN,It's the Most Wonderful Time of the Year,80,0.240,0.598,7,-8.435,1,0.0369,0.766,0.000000,0.1170,0.776,201.629,151933,3
3,46pF1zFimM582ss1PrMy68,Christmas (Baby Please Come Home),66,0.325,0.759,8,-7.146,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,166453,4
4,4PS1e8f2LvuTFgUs1Cn3ON,The Christmas Song (Merry Christmas To You),75,0.320,0.210,6,-15.231,1,0.0345,0.920,0.000080,0.1380,0.208,73.196,192160,5


In [68]:
spotify:track:0bYg9bo50gSsH3LtXe2SQn


In [83]:
audio = pd.DataFrame(sp.audio_features('0bYg9bo50gSsH3LtXe2SQn'))


In [13]:
def get_song_id(name):
    # Search for the track using the song title
    result = sp.search(q=name, type='track', limit=1)

    # Get the song ID from the first result (if available)
    if result['tracks']['items']:
        song_id = result['tracks']['items'][0]['id']
        return song_id
    else:
        print("No matching song found.")

In [14]:
get_song_id("Shape of you")

'7qiZfU4dY1lWllzX7mPBI3'

In [17]:
get_song_id("have yourself a merry little christmas")

'2FPfeYlrbSBR8PwCU0zaqq'